In [1]:
#download training and validation datasets
#training set: model will train on these
#validation set: accuracy of the model will be checked
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip



--2024-05-09 13:11:41--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.207, 142.251.2.207, 2607:f8b0:4023:c06::cf, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip’

horse-or-human.zip  100%[===================>] 142.65M   155MB/s    in 0.9s    

2024-05-09 13:11:42 (155 MB/s) - ‘horse-or-human.zip’ saved [149574867/149574867]

--2024-05-09 13:11:42--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.207, 142.251.2.207, 2607:f8b0:4023:c06::cf, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
L

In [2]:

import zipfile

zip_file = zipfile.ZipFile('./horse-or-human.zip','r')
zip_file.extractall('./horse-or-human')

zip_file = zipfile.ZipFile('./validation-horse-or-human.zip','r')
zip_file.extractall('./validation-horse-or-human')

In [3]:
#Before data is fed into the model, we need to augment it i.e rescale it to a standard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagenerator = ImageDataGenerator(
    rescale=1./255,   #each pixel has value from 0-255
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
    )

validation_datagenerator = ImageDataGenerator(rescale=1./255)

In [4]:
#Now Data Preprocessing:  appropriate labels
TRAINING_DIR = '/content/horse-or-human'
VALIDATION_DIR = '/content/validation-horse-or-human'



train_generator = train_datagenerator.flow_from_directory(
    TRAINING_DIR, # parent directory that contain sub directory for labels
    target_size = (300,300), # make all data sizes uniform
    batch_size = 128, # during each iteration of training, the model will process 128 images from the dataset
    class_mode = 'binary' # we use binary mode because this is a two-class classification problem
)

validation_generator = validation_datagenerator.flow_from_directory(
    VALIDATION_DIR,
    target_size = (300,300),
    batch_size = 32,
    class_mode = 'binary'
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [5]:
# Convulation is applying a filter to extract meaningful features
# Pooling is for reduction and downscaling scale and dimension

#tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', input_shape(300,300,3)) means
# convolution layer with 64 filters, the filters will be 3x3 pixels in size, and the input image shape is 300x300 pixels in 3 color channels (RGB)

#tf.keras.layers.MaxPooling2D(2,2)   means in a 2x2 grid, we pick the pixel with the highest value hence getting highest oriorty->feature extraction

#activation of my layers will be 'relu' except the last one, it will be 'sigmoid' so the output is either 0 or 1

import tensorflow as tf

model = tf.keras.models.Sequential([

    # first convolution and pooling layer
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', input_shape = (300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),

    # second convolution and pooling layer
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # third convolution and pooling layer
    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # forth convolution and pooling layer
    tf.keras.layers.Conv2D(256, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # flatten the output layer to 1D
    tf.keras.layers.Flatten(),

    # add a dropout rate of 0.2
    tf.keras.layers.Dropout(0.2),

    # hidden layer with 128 neuron
    tf.keras.layers.Dense(128, activation='relu'),

    # hidden layer with 512 neuron
    tf.keras.layers.Dense(512, activation='relu'),

    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [7]:
#Let's gt our summary
model.summary();


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 149, 149, 64)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 73, 73, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 35, 35, 128)       0

In [8]:
#The model now exists, let's compile it:
# the problem statemnt is 0/1 based a.k.a binary so loss will be binary crossentropy

from tensorflow.keras.optimizers import RMSprop
#RMSprop adjusts learning rate with each parameter
model.compile(loss = 'binary_crossentropy',
              optimizer = RMSprop(learning_rate = 0.0001),
              metrics = ['accuracy'])


In [ ]:
#on our training set, we will now train the model.
horsehuman_cnn = model.fit(
      train_generator,
      steps_per_epoch=8,
      epochs=25,
      verbose=2,
      validation_data = validation_generator,
      validation_steps=8)

Epoch 1/25


In [ ]:
#Test
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array
import matplotlib.image as mpimg

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = '/content/' + fn
  img = load_img(path, target_size=(300, 300))
  plt.imshow(img)
  plt.axis('off')
  plt.show()

  x = img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=32)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")
  print()


In [ ]:

model.save('my_model.h5')


In [ ]:
from google.colab import files

files.download('my_model.h5')


In [ ]:
#from tensorflow.keras.models import load_model

#model = load_model('my_model.h5')
